In [ ]:
import cv2
import os
import uuid
import time

### image collection

In [ ]:
IMAGE_PATH="/data/images"

In [ ]:
labels=["awake","drowsy"]
number_imgs=20

In [ ]:
for label in labels:
    cap=cv2.VideoCapture(0)
    print("Collecting images for {}".format(label))
    time.sleep(5) #5 secods break before starting
    for imgnum in range(number_imgs):
        print("Collecting imgs {}".format(imgnum))
        ret,frame=cap.read()
        imgname=os.path.join(IMAGE_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname,frame)
        cv2.imshow('frame',frame)
        time.sleep(2)
        
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
PRETRAINED_MODEL_PATH="../tensorflow/pretrained_models"
PRETRAINED_MODEL_NAME="ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
PRETRAINED_MODEL_URL="http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz"

In [ ]:
# pip install wget

#### download object_detection

In [ ]:
!git clone https://github.com/tensorflow/models {"../tensorflow/models"}

#### downloading pretrained model

In [ ]:
import wget
wget.download(PRETRAINED_MODEL_URL)
!move {PRETRAINED_MODEL_NAME+'.tar.gz'} {PRETRAINED_MODEL_PATH} 
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

In [ ]:
# pip install --upgrade protobuf --user

In [ ]:
# pip install --upgrade object_detection --user

### creating label map

In [ ]:
labels=[{'name':'awake','id':1},{'name':'drowsy','id':2}]

with open('../tensorflow/annotations/labelmap.pbtxt','w') as f:
    for label in labels:
        f.write('item{\n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:\'{}\'\n'.format(label['id']))
        f.write('}\n')


### creating tfrecords

In [ ]:
!git clone https://github.com/nicknochnack/GenerateTFRecord {"../tensorflow/scripts"}

In [ ]:
!python {"../tensorflow/scripts/generate_tfrecord.py"} -l="../tensorflow/annotations/labelmap.pbtxt" -x="./data/train" -o="../tensorflow/annotations/train.record"
!python {"../tensorflow/scripts/generate_tfrecord.py"} -l="../tensorflow/annotations/labelmap.pbtxt" -x="./data/test" -o="../tensorflow/annotations/test.record"

In [ ]:
# pip install --upgrade tensorflow --user

### configuring config file

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file("../tensorflow/custom_models/ssd_mobilenet/pipeline.config")

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile("../tensorflow/custom_models/ssd_mobilenet/pipeline.config", "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:

pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 32
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path="../tensorflow/annotations/labelmap.pbtxt"
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join("../tensorflow/annotations", 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = "../tensorflow/annotations/labelmap.pbtxt"
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join("../tensorflow/annotations", 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile("../tensorflow/custom_models/ssd_mobilenet/pipeline.config", "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

#### verification

In [ ]:
VERIFICATION_SCRIPT =  "../tensorflow/models/models/research/object_detection/builders/model_builder_tf2_test.py"
#verify installation
!python {VERIFICATION_SCRIPT}

### train the model

In [ ]:
!python "../tensorflow/models/models/research/object_detection/model_main_tf2.py" --model_dir="../tensorflow/custom_models/ssd_mobilenet" --pipeline_config_path="../tensorflow/custom_models/ssd_mobilenet/pipeline.config" --num_train_steps=1000 verbose=True

### evaluate model

In [ ]:
!python "../tensorflow/models/models/research/object_detection/model_main_tf2.py" --model_dir="./tensorflow/custom_models/ssd_mobilenet" --pipeline_config_path="../tensorflow/custom_models/ssd_mobilenet/pipeline.config" --checkpoint_dir="../tensorflow/custom_models/ssd_mobilenet" 

### loading the model

In [ ]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils  import config_util

In [ ]:
#load pipeline config and build a detection model

configs=config_util.get_configs_from_pipeline_file("../tensorflow/custom_models/ssd_mobilenet/pipeline.config")
detection_model=model_builder.build(model_config=configs['model'],is_training=False)

#restore checkpoint
ckpt=tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join("../tensorflow/custom_models/ssd_mobilenet",'ckpt-3')).assert_existing_objects_matched()
ckpt.restore(os.path.join("../tensorflow/custom_models/ssd_mobilenet",'ckpt-3')).expect_partial()

In [ ]:
def detect_fn(image):
    image,shapes=detection_model.preprocess(image)
    prediction_dict=detection_model.predict(image,shapes)
    detections=detection_model.postprocess(prediction_dict,shapes)
    return detections

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
category_index=label_map_util.create_categories_from_labelmap("../tensorflow/annotations/labelmap.pbtxt")
category_index = {cat['id']: cat for cat in category_index} #converting list to dictionary

In [ ]:
img=cv2.imread("./data/test/awake.eadb4448-151e-11ee-a70d-900f0c7ffb34.jpg")
image_np=np.array(img)
input_tensor=tf.convert_to_tensor(np.expand_dims(image_np,0),dtype=tf.float32)
detections=detect_fn(input_tensor)

num_detections=int(detections.pop('num_detections'))
detections={key:value[0,:num_detections].numpy()
           for key,value in detections.items()}
detections['num_detections']=num_detections

#detection_classes should be ints
detections['detection_classes']=detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

### realtime detection

In [ ]:
cap=cv2.VideoCapture(0)

while cap.isOpened():
    ret,frame=cap.read()
    image_np=np.array(frame)
    
    input_tensor=tf.convert_to_tensor(np.expand_dims(image_np,0),dtype=tf.float32)
    detections=detect_fn(input_tensor)

    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
               for key,value in detections.items()}
    detections['num_detections']=num_detections

    #detection_classes should be ints
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object_detection',cv2.resize(image_np_with_detections,(800,600)))
    
    if cv2.waitKey(1) & 0xFF==ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

### saving the model

In [ ]:
!python "../tensorflow/models/models/research/object_detection/exporter_main_v2.py" --input_type=image_tensor --pipeline_config_path="../tensorflow/custom_models/ssd_mobilenet/pipeline.config" --trained_checkpoint_dir="../tensorflow/custom_models/ssd_mobilenet" --output_directory="./export" 